In [84]:
train['count'][:5]

,count
0,16
1,40
2,32
3,13
4,1


In [94]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# pd.set_option('display.max_rows', None)
# train.shape
# train.info()
# train.describe()
# sns.heatmap(train.isna())

# Convert datetime to datetime type
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

# Extract temporal features
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek

test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['dayofweek'] = test['datetime'].dt.dayofweek

features = ['season', 'holiday', 'workingday', 'weather', 'temp',
            'atemp', 'humidity', 'windspeed', 'hour', 'dayofweek']

X = train[features]
y = train['count']

X_test = test[features]

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data (Important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    layers.InputLayer(input_shape=(X_train_scaled.shape[1],)),  # Input layer
    layers.Dense(64, activation='relu'),  # First hidden layer with ReLU activation
    layers.Dense(32, activation='relu'),  # Second hidden layer with ReLU activation
    layers.Dense(1)  # Output layer with a single node for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mae'])

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val),
                    epochs=50, batch_size=32, verbose=1)

# Evaluate the model on the validation set
val_pred = model.predict(X_val_scaled)
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
val_mae = mean_absolute_error(y_val, val_pred)

print('검증 데이터 RMSE:', val_rmse)
print('검증 데이터 MAE:', val_mae)

# Predict on the test set
test_pred = model.predict(X_test_scaled)

# Clip negative values to zero (or any other value you prefer)
test_pred = np.clip(test_pred, 0, None)  # Clip negative values to zero
test_pred = np.where(test_pred == 0, np.random.randint(1, 4, size=test_pred.shape), test_pred)

# Prepare the submission file
submission = pd.DataFrame({
    'datetime': test['datetime'],
    'count': test_pred.flatten()
})
submission.to_csv('submission.csv', index=False)

print('제출 파일이 생성되었습니다.')


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


273/273 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 63441.0469 - mae: 178.9220 - root_mean_squared_error: 251.4128 - val_loss: 22903.4980 - val_mae: 106.1569 - val_root_mean_squared_error: 151.3390
Epoch 2/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 23000.8555 - mae: 108.4586 - root_mean_squared_error: 151.6390 - val_loss: 21862.2051 - val_mae: 106.8276 - val_root_mean_squared_error: 147.8587
Epoch 3/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 22129.4355 - mae: 108.3570 - root_mean_squared_error: 148.6939 - val_loss: 21568.1406 - val_mae: 107.0408 - val_root_mean_squared_error: 146.8610
Epoch 4/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 21073.7793 - mae: 105.7725 - root_mean_squared_error: 145.1040 - val_loss: 21391.7363 - val_mae: 106.3908 - val_root_mean_squared_error: 146.2591
Epoch 5/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 21860.1895 - mae: 108.0340 - root_mean_squared_error: 147.8361 - val_loss: 21257.1230 - val_mae: 106.0243 - val_root_mean_squar

In [93]:
print(test_pred)

[[  9.781078]
 [ 15.367451]
 [ 16.827415]
 ...
 [ 77.11964 ]
 [ 35.700123]
 [-13.756451]]
